In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow(), utc=True).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
#current_date = pd.Timestamp('2023-11-11 09:00:00+0000', tz='UTC')

current_date=Timestamp('2023-11-11 19:00:00+0000', tz='UTC')


In [22]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-10-14 09:00:00+00:00 to 2023-11-11 08:00:00+00:00
Finished: Reading data from Hopsworks, using ArrowFlight (11.42s) 


In [5]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

In [5]:
predictions['pickup_hour'] = current_date
predictions

,rides_next_1_hour,rides_next_2_hour,rides_next_3_hour,rides_next_4_hour,rides_next_5_hour,rides_next_6_hour,rides_next_7_hour,rides_next_8_hour,rides_next_9_hour,rides_next_10_hour,...,rides_next_29_hour,rides_next_30_hour,rides_next_31_hour,rides_next_32_hour,rides_next_33_hour,rides_next_34_hour,rides_next_35_hour,rides_next_36_hour,pickup_location_id,pickup_hour
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,0.0,1.0,2,2023-11-11 09:00:00+00:00
1,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,3,2023-11-11 09:00:00+00:00
2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,4,2023-11-11 09:00:00+00:00
3,2.0,3.0,3.0,4.0,4.0,4.0,5.0,4.0,5.0,7.0,...,4.0,3.0,5.0,5.0,8.0,8.0,4.0,3.0,5,2023-11-11 09:00:00+00:00
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,6,2023-11-11 09:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,493,2023-11-11 09:00:00+00:00
315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,494,2023-11-11 09:00:00+00:00
316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496,2023-11-11 09:00:00+00:00
317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,497,2023-11-11 09:00:00+00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/100501/fs/100420/fg/188782


Uploading Dataframe: 0.00% |          | Rows 0/319 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/100501/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f348c265e0>, None)